In [1]:
from matplotlib import pyplot as plt
from sacred import Experiment
import tensorflow as tf

from bfseg.cl_models import BaseCLModel
from bfseg.utils.datasets import load_data
from bfseg.utils.images import augmentation
from bfseg.utils.losses import IgnorantCrossEntropyLoss
from bfseg.utils.models import create_model

ex = Experiment('jupyter_ex', interactive=True)

@ex.config
def default_config():
  r"""Default configuration for fine-tuning experiment on NYU with U-Net.
  """
  # Network parameters.
  network_params = {
      'architecture': 'fast_scnn',
      'freeze_encoder': False,
      'freeze_whole_model': False,
      'normalization_type': "group",
      'model_params': {
          'image_h': 480,
          'image_w': 640
      }
  }

  # Training parameters.
  training_params = {
      'batch_size': 8,
      'learning_rate': 1e-4,
      'num_training_epochs': 1,
      'stopping_patience': 100
  }

  # Dataset parameters.
  dataset_params = {
      'test_dataset': "NyuDepthV2Labeled",
      'test_scene': None,
      'train_dataset': "BfsegCLAMeshdistLabels",
      'train_scene': None,
      'validation_percentage': 20
  }

  # Logging parameters.
  logging_params = {
      'model_save_freq': 1,
      'exp_name': "exp_stage1"
  }

  # CL parameters.
  cl_params = {'cl_framework': "finetune", 'pretrained_dir': None}

def main():
    model = BaseCLModel(ex.current_run, '/home/fmilano/tmp')

    batch_size = 8

    train_ds = load_data(dataset_name='BfsegCLAMeshdistLabels',
                         scene_type=None,
                         fraction="[:5%]",
                         batch_size=batch_size,
                         shuffle_data=False)

    model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
                  loss = IgnorantCrossEntropyLoss(class_to_ignore=2,
                                                  num_classes=3,
                                                  from_logits=True))
    model.fit(train_ds.map(augmentation), epochs=1)
    
    return model

Segmentation Models: using `tf.keras` framework.


In [2]:
ex.main(main)

<function __main__.main()>

In [3]:
batch_size = 8

train_ds = load_data(dataset_name='BfsegCLAMeshdistLabels',
     scene_type=None,
     fraction="[:5%]",
     batch_size=batch_size,
     shuffle_data=False)

for sample_1 in train_ds.take(1):
    break
    
for sample_2 in train_ds.unbatch().map(augmentation).batch(batch_size).take(1):
    break

In [4]:
sample_1[2].shape

TensorShape([8, 480, 640])

In [5]:
sample_2[2].shape

TensorShape([8, 480, 640])

In [6]:
total_len = sum(a[0].shape[0] for a in train_ds)

In [7]:
total_len

135

In [8]:
import numpy as np

In [9]:
for i, s in enumerate(train_ds.unbatch().shuffle(total_len).batch(5).take(3)):
    print(i)
    print(s[0].shape)

0
(5, 480, 640, 3)
1
(5, 480, 640, 3)
2
(5, 480, 640, 3)


In [10]:
app_ds = train_ds.unbatch().batch(1).take(3)
for i, s in enumerate(app_ds):
    print(i)
    print(s[0].shape)

0
(1, 480, 640, 3)
1
(1, 480, 640, 3)
2
(1, 480, 640, 3)


In [11]:
app_ds_1 = train_ds.unbatch().shuffle(total_len).batch(3).take(5)
app_ds_2 = train_ds.unbatch().shuffle(total_len).batch(2).take(5)

In [12]:
app_ds_1

<TakeDataset shapes: ((None, 480, 640, 3), (None, 480, 640, 1), (None, 480, 640)), types: (tf.float32, tf.uint8, tf.bool)>

In [13]:
for s_1 in app_ds_1.take(1):
    break
for s_2 in app_ds_2.take(1):
    break

In [14]:
s_2[0].dtype

tf.float32

In [15]:
tf.concat((s_1[0], s_2[0]), axis=0)

<tf.Tensor: shape=(5, 480, 640, 3), dtype=float32, numpy=
array([[[[0.16462524, 0.15694037, 0.16517109],
         [0.16469373, 0.15739992, 0.16052428],
         [0.16899016, 0.16400999, 0.14731735],
         ...,
         [0.37104237, 0.37542456, 0.31680372],
         [0.36108556, 0.36530396, 0.34727114],
         [0.34878522, 0.3647805 , 0.3870189 ]],

        [[0.16040283, 0.16194853, 0.19108596],
         [0.15273869, 0.1623808 , 0.18160787],
         [0.14946257, 0.1681771 , 0.13946353],
         ...,
         [0.35197255, 0.36036327, 0.3393286 ],
         [0.34178394, 0.3602777 , 0.36260355],
         [0.32739463, 0.3670954 , 0.36469224]],

        [[0.15642083, 0.17886262, 0.17378521],
         [0.15392436, 0.18007231, 0.17385246],
         [0.15694365, 0.1800573 , 0.14592718],
         ...,
         [0.3315354 , 0.3418717 , 0.38570535],
         [0.34238204, 0.35190925, 0.35740498],
         [0.35389078, 0.34410217, 0.3687146 ]],

        ...,

        [[0.24809273, 0.19964544, 

In [16]:
def concat_samples(sample_1, sample_2):
    return (tf.concat((sample_1[0], sample_2[0]), axis=0),
            tf.concat((sample_1[1], sample_2[1]), axis=0),
            tf.concat((sample_1[2], sample_2[2]), axis=0))

In [17]:
merged_ds = tf.data.Dataset.zip((app_ds_1, app_ds_2)).map(concat_samples)

In [18]:
for ms_1 in merged_ds.take(1):
    break

In [19]:
ms_1[0].shape

TensorShape([5, 480, 640, 3])

In [20]:
concat_ds = train_ds.concatenate(app_ds_1).batch

In [21]:
from bfseg.utils.replay_buffer import ReplayBuffer

In [22]:
replay_buffer = ReplayBuffer(main_ds=train_ds, replay_ds=app_ds_1, batch_size=6, ratio_main_ds_replay_ds=[20, 1])

In [23]:
replay_buffer._tot_num_samples_main

135

In [24]:
replay_buffer._tot_num_samples_replay

15

In [25]:
merged_ds_1 = replay_buffer.flow()

In [26]:
for sample_idx, sample in enumerate(merged_ds_1):
    print(sample_idx)
    print(sample[0].shape)
    if (sample_idx in [0, 1]):
        print(sample[0][0, :10, :2])

0
(6, 480, 640, 3)
tf.Tensor(
[[[0.7375212  0.69755363 0.7936749 ]
  [0.79827476 0.80070025 0.80913085]]

 [[0.7357196  0.7023351  0.7885317 ]
  [0.80803484 0.80866915 0.8089116 ]]

 [[0.621825   0.68053335 0.76674443]
  [0.78482664 0.7977483  0.8047084 ]]

 [[0.56405383 0.62340844 0.77084666]
  [0.7231334  0.75516176 0.7970326 ]]

 [[0.52504086 0.56235373 0.7917612 ]
  [0.65581906 0.67951494 0.79194623]]

 [[0.49376047 0.555717   0.74311805]
  [0.6012308  0.62357265 0.7954646 ]]

 [[0.44711816 0.5185996  0.6415843 ]
  [0.5919628  0.64506614 0.7716049 ]]

 [[0.43922597 0.4799979  0.58139914]
  [0.5380694  0.6378627  0.7035286 ]]

 [[0.43240392 0.4400888  0.54535764]
  [0.5317327  0.5772163  0.6322798 ]]

 [[0.45152295 0.46172243 0.5210068 ]
  [0.51289916 0.52503306 0.57762647]]], shape=(10, 2, 3), dtype=float32)
1
(6, 480, 640, 3)
tf.Tensor(
[[[0.20924138 0.19869444 0.21421015]
  [0.1990427  0.19910371 0.20253633]]

 [[0.19371785 0.19641852 0.2150966 ]
  [0.19147222 0.19998035 0.218662

In [27]:
replay_buffer_2 = ReplayBuffer(main_ds=train_ds, replay_ds=app_ds_1, batch_size=6, fraction_replay_ds_to_use=0.4)

In [28]:
replay_buffer_2._tot_num_samples_main

135

In [29]:
replay_buffer_2._tot_num_samples_replay

15

In [30]:
merged_ds_2 = replay_buffer_2.flow()

In [32]:
for app_sample_1 in app_ds_1:
    print(app_sample_1[0].shape)

(3, 480, 640, 3)
(3, 480, 640, 3)
(3, 480, 640, 3)
(3, 480, 640, 3)
(3, 480, 640, 3)


In [33]:
for sample_idx, sample in enumerate(merged_ds_2):
    print(sample_idx)
    print(sample[0].shape)

0
(6, 480, 640, 3)
1
(6, 480, 640, 3)
2
(6, 480, 640, 3)
3
(6, 480, 640, 3)
4
(6, 480, 640, 3)
5
(6, 480, 640, 3)
6
(6, 480, 640, 3)
7
(6, 480, 640, 3)
8
(6, 480, 640, 3)
9
(6, 480, 640, 3)
10
(6, 480, 640, 3)
11
(6, 480, 640, 3)
12
(6, 480, 640, 3)
13
(6, 480, 640, 3)
14
(6, 480, 640, 3)
15
(6, 480, 640, 3)
16
(6, 480, 640, 3)
17
(6, 480, 640, 3)
18
(6, 480, 640, 3)
19
(6, 480, 640, 3)
20
(6, 480, 640, 3)
21
(6, 480, 640, 3)
22
(6, 480, 640, 3)
23
(3, 480, 640, 3)
